In [97]:
import hoomd
import gsd.hoomd
import numpy
import numpy as np
import math

import itertools
import matplotlib.pyplot as plt

In [98]:
N_particles=200

spacing = 2
K = math.ceil(N_particles**(1 / 3))
L = K * spacing
x = numpy.linspace(-L / 2, L / 2, K, endpoint=False)
position = list(itertools.product(x, repeat=3))

frame = gsd.hoomd.Frame()
frame.particles.N = N_particles
frame.particles.position = position[0:N_particles]
frame.particles.typeid = [0] * N_particles
frame.particles.types = ['A']
frame.particles.mass=[5]*N_particles
frame.configuration.box = [L, L, L, 0, 0, 0]
a=frame

In [99]:
test = [np.random.uniform(-10,10,size=3) for _ in range(200)]

In [100]:
tau=1
kT=.1
dt=.0005
epsilon=2
frames=3e5
trigger=1000

In [101]:
#Script in order to account for Intertia, which hopefully solves the line 9 problem of providing no torques
#The problem: adding the mass property to the particles breaks the line 4 command
general_positions = numpy.array([[0, 0, 0], [1, 0, 0]])

I_ref = numpy.array([[0.4, 0, 0],
                   [0, 0.4, 0],
                   [0, 0, 0.4]])
I_general = numpy.zeros(shape=(3,3))
for r in general_positions:
    I_general += I_ref + 1 * (numpy.dot(r, r) * numpy.identity(3) - numpy.outer(r, r))
    
I_diagonal, E_vec = numpy.linalg.eig(I_general)

R = E_vec.T

diagonal_positions = numpy.dot(R, general_positions.T).T

I_check = numpy.zeros(shape=(3,3))
for r in diagonal_positions:
    I_check += I_ref + 1 * (numpy.dot(r, r) * numpy.identity(3) - numpy.outer(r, r))

#mass = 1
#I = numpy.zeros(shape=(3,3))
#for r in central_position:
#    I += mass * (numpy.dot(r, r) * numpy.identity(3) - numpy.outer(r, r))
I_check

array([[0.8, 0. , 0. ],
       [0. , 1.8, 0. ],
       [0. , 0. , 1.8]])

In [102]:
a.particles.moment_inertia = [I_general[0, 0], I_general[1, 1], I_general[2, 2]] * 200
print(a.particles.moment_inertia[:2])

[0.8, 1.7999999999999998]


In [103]:
device = hoomd.device.auto_select()
sim = hoomd.Simulation(device=device, seed=1)
sim.create_state_from_snapshot(a)

In [104]:
rigid = hoomd.md.constrain.Rigid()

rigid.body['dimer'] = {
    "constituent_types": ['A', 'A'],
    "positions": [[0, 0, -0.5], [0, 0, 0.5]],
    "orientations": [(1.0, 0.0, 0.0, 0.0), (1.0, 0, 0, 1.0)],
    "charges": [0.0, 0.0],
    "diameters": [1.0, 1.0]
}

In [105]:
integrator = hoomd.md.Integrator(dt=dt, integrate_rotational_dof=True)

cell=hoomd.md.nlist.Cell(buffer=0.4)

lperp=0.3
lpar=1.0
sigmin=2*min(lperp,lpar)
sigmax=2*max(lperp,lpar)

gay_berne=hoomd.md.pair.aniso.GayBerne(nlist=cell, default_r_cut=2.5)
gay_berne.params[('A','A')]=dict(epsilon=epsilon, lperp=lperp, lpar=lpar)
gay_berne.r_cut[('A','A')]=2.5

In [106]:
sim.operations.integrator=integrator

In [107]:
integrator.rigid=rigid

In [108]:
rigid_centers_and_free=hoomd.filter.Rigid(("center","free"))
nvt=hoomd.md.methods.NVT(filter=hoomd.filter.All(),kT=kT,tau=tau)
#nve=hoomd.md.methods.NVE(filter=hoomd.filter.All())
integrator.methods.append(nvt)
integrator.forces.append(gay_berne)

In [109]:
sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=kT)
sim.run(0)

In [110]:
logger=hoomd.logging.Logger(categories=['scalar'])
logger+=gay_berne
logger.add(gay_berne,quantities=['energy'])

In [111]:
Table = hoomd.write.Table(output=open('gay_berne_log_different_directions.txt', mode='w', newline='\n'),
                          trigger = hoomd.trigger.Periodic(1), logger=logger)
sim.operations.writers.append(Table)

In [112]:
sim.run(0)

In [113]:
#g=sim.state.get_snapshot()
#g.particles.orientation

In [114]:
thermodynamic_properties=hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())
gsd_writer=hoomd.write.GSD(filename='traj_different_directions1.gsd',trigger=hoomd.trigger.Periodic(trigger),mode='wb',filter=hoomd.filter.All())
sim.operations.writers.append(gsd_writer)

In [115]:
#ramp = hoomd.variant.Ramp(A=0, B=1, t_start=sim.timestep, t_ramp=20000)
#initial_box = sim.state.box
#final_box = sim.state.box
#final_box.Lx *= 0.80
#final_box.Ly *= 0.80
#final_box.Lz *= 0.80
#box_resize_trigger = hoomd.trigger.Periodic(trigger)
#box_resize = hoomd.update.BoxResize(box1=initial_box,
#                                    box2=final_box,
#                                    variant=ramp,
#                                    trigger=box_resize_trigger)
#sim.operations.updaters.append(box_resize)

In [116]:
sim.state.box.Lx *= 1

In [117]:
sim.operations.computes.append(thermodynamic_properties)
sim.run(frames,write_at_start=True)

In [118]:
sim.timestep

300000

In [119]:
sim.tps

2978.140596925596

In [120]:
traj = gsd.hoomd.open("traj_different_directions1.gsd")

In [121]:
len(traj)

301